In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the forward pass. Then, during the backward pass, TensorFlow traverses this list of operations in reverse order to compute gradients.

[Gradient tapes](https://www.tensorflow.org/api_docs/python/tf/GradientTape)

In [3]:
# Gradient tapes

w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

[dl_dw, dl_db] = tape.gradient(loss, [w, b])

# The gradient w.r.t. each source has the shape of the source:
print(w.shape)
print(dl_dw.shape)

(3, 2)
(3, 2)


Gradients w.r.t. a model